# Flappy Bird

In [1]:
# @ title Imports
from time import sleep

import numpy as np
from IPython.display import clear_output
from deep_rl.environments.flappy_bird import FlappyBird
from deep_rl.shapes import Point, Rectangle
from deep_rl.terminal_renderer import BashRenderer
from typing import Tuple
from src import (
    infer_parameters,
    TreeBuilder,
    get_best_action,
    print_outcomes_stats,
    repr_obs,
)

In [2]:
params = {"gravity": 0.05, "force_push": 0.1, "vx": 0.05}
env = FlappyBird(**params, prob_new_bar=1, max_height_bar=0.5)

In [3]:
# @ title Inferring the parameters
gravity, force_push, vx = infer_parameters(env)
assert np.allclose(
    [gravity, force_push, vx],
    (
        params["gravity"],
        params["force_push"],
        params["vx"],
    ),
    rtol=1e-3,
), "Parameters inference failed"

In [4]:
# @title Building the tree
env.reset()
(bird_x, bird_y, bird_vy), bars = env.step(0)[0]
tree_builder = TreeBuilder(bars, gravity, force_push, vx)
outcomes = tree_builder.build_tree(bird_x, bird_y, bird_vy)
print(
    f"\nNumber of leaves computed: {tree_builder.n_steps_computed}\n"
    f"Number of leave computation steps saved: {tree_builder.n_steps_saved}\n"
)
print_outcomes_stats(outcomes)


Number of leaves computed: 628
Number of leave computation steps saved: 3468


Number of favorable outcomes:    547 / 4096  (13.35%)
- Probability of winning when standing still: 4.79%
- Probability of winning when jumping:        8.57%



## Experiments

In [12]:
def launch_experiment(
    predicted_outcomes: np.ndarray,
    rendering_size: Tuple[int, int] = (20, 40),
    time_between_frames: float = 0.1,
    clear_between_frames: bool = False,
    max_steps: int = 1000,
) -> Tuple[int, int]:
    step, total_reward = 0, 0
    renderer = BashRenderer(*rendering_size, clear_fn=lambda: clear_output(wait=True))

    for _ in range(max_steps):
        action = get_best_action(predicted_outcomes)
        observation, reward, done = env.step(action)
        print(
            f"action: {action}, reward: {reward}, observation: {repr_obs(observation)}"
        )

        step += 1
        total_reward += reward

        print(f"Cumulated reward at step {step}: {total_reward:>3}.")
        if done:
            print(f"Simulation ended after {step} steps.")
            break
        tree_builder = TreeBuilder(observation[1], gravity, force_push, vx)
        predicted_outcomes = tree_builder.build_tree(*observation[0])
        print_outcomes_stats(predicted_outcomes)

        if clear_between_frames:
            renderer.clear()
        renderer.draw_list(env.render())
        renderer.draw_title(f"Total reward : {total_reward}")
        renderer.render()
        sleep(time_between_frames)

        background = Rectangle(
            bot_left=Point(x=0, y=0),
            top_right=Point(x=1, y=1),
        )
    return total_reward, step

In [37]:
launch_experiment(outcomes)

action: 1, reward: 0, observation: ((0.5, 0.4, 0.05), [(0.29999999999999966, 0.3999999999999997, 0.19332566575340476, False), (0.34999999999999964, 0.4499999999999996, 0.13047509607014723, True), (0.8499999999999999, 0.9499999999999998, 0.08287234753870709, True), (0.95, 1.05, 0.1060008283934924, True), (0.8999999999999999, 0.9999999999999999, 0.07056725427566568, False), (0.5999999999999996, 0.6999999999999996, 0.27647398582211, True)])
Cumulated reward at step 1:   0.

Number of favorable outcomes:    549 / 2048  (26.81%)
- Probability of winning when standing still: 17.19%
- Probability of winning when jumping:        9.62%

　　　　　　　　　　　🟩🟩🟩🟩🟩　　　　　　　　　　　　　　　　　　　　🟩🟩🟩🟩
　　　　　　　　　　　🟩🟩🟩🟩🟩　　　　　　　　　　　　　　　　　　　　🟩🟩🟩🟩
　　　　　　　　　　　🟩🟩🟩🟩🟩　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　🟩🟩🟩🟩🟩　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　

In [14]:
launch_experiment(outcomes)

action: 1, reward: 0, observation: (0.50, 0.55, 0.05)
 - (1.00, 1.10, 0.36, TOP)
Cumulated reward at step 1:   0.

Number of favorable outcomes:    590 / 4096  (14.40%)
- Probability of winning when standing still: 10.13%
- Probability of winning when jumping:        4.27%

　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　🐤　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
　　　　　　　　　　　　　　　　　　　　　　　　　　　　

(-1, 13)